In [77]:
# import the necessary packages
# 1. SQL Integration
import mysql.connector
from mysql.connector import errorcode
# 2. Computer Vision
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import imutils
import dlib
import cv2
# 3. Analysis
from scipy.spatial import distance as dist
import numpy as np
import time
from datetime import datetime

In [78]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "___",
    database = 'capstone'
)

In [79]:
print(mydb)

In [80]:
cursor = mydb.cursor()

In [81]:
#cursor.execute('CREATE TABLE brLog(timestamp float, blink_count int, blink_rate float, primary key(timestamp))')

In [82]:
hog_face_detector = dlib.get_frontal_face_detector()

In [83]:
dlib_facelandmark = dlib.shape_predictor("D:/Study/Uni/1. Capstone II/PyCharm Codes/shape_predictor_68_face_landmarks.dat")

In [84]:
cap = cv2.VideoCapture(0)

In [85]:
# Test code for facial detection
"""
while True:
    _, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = hog_face_detector(gray)
    for face in faces:

        face_landmarks = dlib_facelandmark(gray, face)

        for n in range(0, 68):
            x = face_landmarks.part(n).x
            y = face_landmarks.part(n).y
            cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)


    cv2.imshow("Face Landmarks", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()
"""

'\nwhile True:\n    _, frame = cap.read()\n    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)\n\n    faces = hog_face_detector(gray)\n    for face in faces:\n\n        face_landmarks = dlib_facelandmark(gray, face)\n\n        for n in range(0, 68):\n            x = face_landmarks.part(n).x\n            y = face_landmarks.part(n).y\n            cv2.circle(frame, (x, y), 1, (0, 255, 255), 1)\n\n\n    cv2.imshow("Face Landmarks", frame)\n\n    key = cv2.waitKey(1)\n    if key == 27:\n        break\ncap.release()\ncv2.destroyAllWindows()\n'

In [86]:
def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear

In [87]:
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3

# initialize the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0

In [88]:
# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [89]:
start_time = time.time()
tick = 2
while True:
    current_time = time.time()
    duration = current_time - start_time
    _, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = hog_face_detector(gray)
    for face in faces:

        face_landmarks = face_utils.shape_to_np(dlib_facelandmark(gray, face))

        leftEye = face_landmarks[lStart:lEnd]
        rightEye = face_landmarks[rStart:rEnd]
        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)
        
        # average the eye aspect ratio together for both eyes
        ear = (leftEAR + rightEAR) / 2
        # compute the convex hull for the left and right eye, then
        # visualize each of the eyes
        leftEyeHull = cv2.convexHull(leftEye)
        rightEyeHull = cv2.convexHull(rightEye)
        cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
        
        # check to see if the eye aspect ratio is below the blink
        # threshold, and if so, increment the blink frame counter
        if ear < EYE_AR_THRESH:
            COUNTER += 1
            
        # otherwise, the eye aspect ratio is not below the blink
        # threshold
        else:
            # if the eyes were closed for a sufficient number of
            # then increment the total number of blinks
            if COUNTER >= EYE_AR_CONSEC_FRAMES:
                TOTAL += 1
                
            # reset the eye frame counter
            COUNTER = 0
        
        blink_rate = 0
        if duration != float(0):
            blink_rate = 60*TOTAL/duration
        
        if int(duration) == tick:
            tick += 2
            cursor.execute('insert into brLog values (%s,%s,%s)', (duration, TOTAL, blink_rate))            
            mydb.commit()

        # draw the total number of blinks on the frame along with
        # the computed eye aspect ratio for the frame
        cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Duration: {:.2f}".format(duration), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, "Blink rate: {:.2f}".format(blink_rate), (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Face Landmarks", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()